In [ ]:
from tqdm import tqdm
import random
import pandas as pd
import numpy as np

In [ ]:
# import train
train = {}
with open('train.txt') as f: 
    for line in f:
        words = line.split("\t")
        words[-1] = words[-1][:-1]
        train[words[0]] = words[1:]

In [ ]:
# import the sources and sinks
test_source = []
test_sink = []
with open('test-public.txt') as f: 
    for line in f:
        words = line.split("\t")
        test_source.append(words[1])
        test_sink.append(words[2][:-1])
    test_source = test_source[1:]
    test_sink= test_sink[1:]

In [ ]:
# get the people that the sources follow
source_followees = {}
for source in test_source:
    source_followees[source] = train[source]

# get the people that follow the sources
source_followers = {}
for source in tqdm(test_source):
    source_followers[source] = []
    for key in train.keys():
        if source in train[key]:
            source_followers[source].append(key)
            
# put the source information in one dict
Source = {}
for source in test_source:
    Source[source] = {}
    Source[source]["followees"] = source_followees[source]
    Source[source]["n_followees"] = len(source_followees[source])
    Source[source]["followers"] = source_followers[source]
    Source[source]["n_followers"] = len(source_followers[source])
    Source[source]["n_neighbours"] = Source[source]["n_followees"] + Source[source]["n_followers"]

In [ ]:
# export Source dict as a txt
import json
with open('Source_information.txt', 'w') as file:
     file.write(json.dumps(Source))

In [ ]:
# get the people that follow the sinks
sink_followers = {}
for sink in tqdm(test_sink):
    sink_followers[sink] = []
    for key in train:
        if sink in train[key]:
            sink_followers[sink].append(key)
            
# get the people that the sinks follow
sink_followees = {}
for sink in test_sink:
    if sink in train.keys():
        sink_followees[sink] = train[sink]
        
# get the people that are followed by the ones who follow the sinks
sink_followers_followees = {}
for sink in tqdm(test_sink):
    sink_followers_followees[sink] = {}
    for sink_follower in sink_followers[sink]:
        sink_followers_followees[sink][sink_follower] = train[sink_follower]

# put the sink information in one dict
Sink = {}
for sink in test_sink:
    Sink[sink] = {}
    if sink in train.keys():
        Sink[sink]["followees"] = sink_followees[sink]
        Sink[sink]["n_followees"] = len(sink_followees[sink])
    else:
        Sink[sink]["followees"] = []
        Sink[sink]["n_followees"] = 0
    Sink[sink]["followers"] = sink_followers[sink]
    Sink[sink]["n_followers"] = len(sink_followers[sink])
    Sink[sink]["n_neighbours"] = Sink[sink]["n_followees"] + Sink[sink]["n_followers"]
    Sink[sink]["followers_followees"] = {}
    for i in sink_followers_followees[sink].keys():
        Sink[sink]["followers_followees"][i] = {}
        Sink[sink]["followers_followees"][i]["ID"] = sink_followers_followees[sink][i]
        Sink[sink]["followers_followees"][i]["n"] = len(sink_followers_followees[sink][i])

In [ ]:
# export Sink dict as a txt
import json
with open('Sink_information.txt', 'w') as file:
     file.write(json.dumps(Sink))

In [ ]:
# import the source_information
import ast
file = open("Source_information.txt", "r")

contents = file.read()
Source = ast.literal_eval(contents)

file.close()

In [ ]:
len(Sink)

In [ ]:
source = test_source[0]
sink = test_sink[0]
similarities = []

In [ ]:
# calculate the proportion of source's followees that appear in sink's followers' folowees

for sink_follower_followee in tqdm(Sink[sink]['followers_followees']):
    intersect = set.intersection(set(Source[source]["followees"]),
                                         set(Sink[sink]['followers_followees']
                                             [sink_follower_followee]['ID']))
    union = set(Source[source]["followees"])|set(Sink[sink]['followers_followees']
                                             [sink_follower_followee]['ID'])
    similarities.append(len(intersect)/len(union))

In [ ]:
np.mean(similarities)

In [ ]:
common_interest = []
for i in tqdm(range(len(test_source))):
    source = test_source[i]
    sink = test_sink[i]
    similarities = []
    for sink_follower_followee in Sink[sink]['followers_followees']:
        intersect = set.intersection(set(Source[source]["followees"]),
                                             set(Sink[sink]['followers_followees']
                                                 [sink_follower_followee]['ID']))
        union = set(Source[source]["followees"])|set(Sink[sink]['followers_followees']
                                                 [sink_follower_followee]['ID'])
        if len(union) == 0:
            similaries.append(0)
        else:
            similarities.append(len(intersect)/len(union))
    common_interest.append(np.mean(similarities))

In [ ]:
import scipy.stats as ss
common_interest_feature = ss.rankdata(common_interest)/2000

In [ ]:
# export test_mini edges
test_common_interest = pd.DataFrame(test_source)
test_common_interest.insert(1,"test_sink",test_sink)
test_common_interest.insert(2,"common_interest",common_interest)
test_common_interest = test_common_interest.rename(columns={0: 'test_source'})
test_common_interest.to_csv("test_common_interest_2.csv")